In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint


### Dicionário das variáveis:

Temperature (K): 

Luminosity(L/Lo):

Radius(R/Ro):

Absolute magnitude(Mv):

*Star Type*: Brown Dwarf = 0, Red Dwarf = 1, White Dwarf = 2, Main Sequence = 3, Supergiant = 4, Hypergiant = 5

Star color:

Spectral Class:

The Luminosity and radius of each star is calculated w.r.t. that of the values of Sun.

Lo = 3.828 x 10^26 Watts

Ro = 6.9551 x 10^8 m

In [2]:
dados = pd.read_csv('stars.csv')

In [3]:
dados.shape

(240, 7)

In [4]:
dados.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Temperature (K)         240 non-null    int64  
 1   Luminosity(L/Lo)        240 non-null    float64
 2   Radius(R/Ro)            240 non-null    float64
 3   Absolute magnitude(Mv)  240 non-null    float64
 4   Star type               240 non-null    int64  
 5   Star color              240 non-null    object 
 6   Spectral Class          240 non-null    object 
dtypes: float64(3), int64(2), object(2)
memory usage: 13.2+ KB


In [6]:
dados.columns

Index(['Temperature (K)', 'Luminosity(L/Lo)', 'Radius(R/Ro)',
       'Absolute magnitude(Mv)', 'Star type', 'Star color', 'Spectral Class'],
      dtype='object')

In [7]:
numericas = ['Temperature (K)', 'Luminosity(L/Lo)', 'Radius(R/Ro)',
       'Absolute magnitude(Mv)', 'Star type']

categoricas = ['Star color', 'Spectral Class']

In [8]:
dados[numericas].describe()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type
count,240.000000,240.000000,240.000000,240.000000,240.000000
mean,10497.462500,107188.361635,237.157781,4.382396,2.500000
std,9552.425037,179432.244940,517.155763,10.532512,1.711394
min,1939.000000,0.000080,0.008400,-11.920000,0.000000
25%,3344.250000,0.000865,0.102750,-6.232500,1.000000
50%,5776.000000,0.070500,0.762500,8.313000,2.500000
75%,15055.500000,198050.000000,42.750000,13.697500,4.000000
max,40000.000000,849420.000000,1948.500000,20.060000,5.000000


In [9]:
dados[categoricas].describe()

,Star color,Spectral Class
count,240,240
unique,19,7
top,Red,M
freq,112,111


In [10]:
dados['Star color'].unique()

array(['Red', 'Blue White', 'White', 'Yellowish White', 'Blue white',
       'Pale yellow orange', 'Blue', 'Blue-white', 'Whitish',
       'yellow-white', 'Orange', 'White-Yellow', 'white', 'Blue ',
       'yellowish', 'Yellowish', 'Orange-Red', 'Blue white ',
       'Blue-White'], dtype=object)

In [11]:
dados['Spectral Class'].unique()

array(['M', 'B', 'A', 'F', 'O', 'K', 'G'], dtype=object)

# Separando os dados entre teste e treino

In [14]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))
    
    indices = df.index.tolist()
    test_indices = random.sample(population = indices, k = test_size)
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [15]:
random.seed(0)
treino, teste = train_test_split(dados,0.1)

In [17]:
treino.shape, teste.shape

((216, 7), (24, 7))

# Funçoes para criação da árvore de decisão

Importante! A variável resposta deve ser a última coluna do data frame.

In [18]:
data = treino.values

### Data pure?

In [21]:
def check_purity(data):
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else: 
        return False

In [22]:
check_purity(treino.values)

False

In [24]:
check_purity(treino.head().values)

True

### Classificador

In [30]:
def classify_data(data):
    
    label_column = data[:,-1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts = True)
    
    index = counts_unique_classes.argmax()
    
    classification = unique_classes[index]
    
    return classification

### Potencial split

In [51]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
                
    return potential_splits

### Split data

In [32]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]
    
    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values > split_value]
    
    return data_below, data_above

In [33]:
split_column = 3
split_value = 0.8

In [35]:
data_below, data_above = split_data(data, split_column, split_value)

### Lowest Overall Impurity? Gini and entropy

In [36]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts = True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [37]:
def calculate_gini(data):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts = True)
    
    probabilities = (counts / counts.sum())**2
    
    gini = 1 - sum(probabilities)
    
    return gini

In [38]:
def calculate_overall_entropy(data_below, data_above):
    n_data_points = len(data_below) + len(data_above)
    
    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points
    
    overall_entropy = (p_data_below * calculate_entropy(data_below)) + (p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [39]:
def calculate_overall_gini(data_below, data_above):
    n_data_points = len(data_below) + len(data_above)
    
    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points
    
    overall_gini = (p_data_below * calculate_gini(data_below)) + (p_data_above * calculate_gini(data_above))
    
    return overall_gini

In [40]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column = column_index, split_value = value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            
            if current_overall_entropy < overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [41]:
def determine_best_split(data, potential_splits, impurity = 'gini'):
    
    overall_impurity = 999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column = column_index, split_value = value)
            if impurity == 'entropy':
                current_overall_impurity = calculate_overall_entropy(data_below, data_above)
            else: # impurity == 'gini'
                current_overall_impurity = calculate_overall_gini(data_below, data_above)
            
            if current_overall_impurity < overall_impurity:
                overall_impurity = current_overall_impurity
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

## Função principal 

In [1]:
def decision_tree_algorithm(df, counter = 0, min_samples = 2, impurity = 'gini'):
    
    # 
    if counter == 0:
        global column_headers
        column_headers = df.columns
        data = df.values
    else:
        data = df
        
    #base case
    if (check_purity(data)) or (len(data) < min_samples):
        classification = classify_data(data)
        return classification
    
    #recursive part
    else:
        counter += 1
        
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits, impurity = 'gini')
        data_below, data_above = split_data(data,split_column, split_value)
        
        # instantiate sub-tree
        feature_name = column_headers[split_column]
        question = "{} <= {:.6f}".format(feature_name, split_value)
        sub_tree = {question: []}
    
        #find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, impurity)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples. impurity)
        
        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)
        
    return sub_tree

In [2]:
tree = decision_tree_algorithm(dados.drop('Star color', axis = 1))

NameError: name 'dados' is not defined

In [77]:
pprint(tree)

{'Temperature (K) <= 3924.500000': ['M',
                                    {'Absolute magnitude(Mv) <= -4.280000': [{'Radius(R/Ro) <= 403.450000': [{'Temperature (K) <= 32880.000000': ['O',
                                                                                                                                                  {'Temperature (K) <= 33970.000000': [{'Luminosity(L/Lo) <= 296000.000000': ['B',
                                                                                                                                                                                                                              'O']},
                                                                                                                                                                                       'O']}]},
                                                                                                             {'Temperature (K) <= 29289.000000': [{'Temperature 

In [57]:
dados.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M


In [60]:
dados[dados['Temperature (K)'] <= 3924.5]['Spectral Class'].unique()

array(['M'], dtype=object)